In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [6]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [7]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.55113, saving model to best_m.h5
 - 3s - loss: 0.8450 - acc: 0.6431 - val_loss: 0.5511 - val_acc: 0.7382
Epoch 2/50
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.4756 - acc: 0.7759 - val_loss: 0.6745 - val_acc: 0.7382
Epoch 3/50
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4371 - acc: 0.7941 - val_loss: 0.5541 - val_acc: 0.7357
Epoch 4/50
Epoch 00004: val_loss improved from 0.55113 to 0.37658, saving model to best_m.h5
 - 2s - loss: 0.4004 - acc: 0.8066 - val_loss: 0.3766 - val_acc: 0.8354
Epoch 5/50
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.3614 - acc: 0.8383 - val_loss: 0.7340 - val_acc: 0.7207
Epoch 6/50
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.3588 - acc: 0.8416 - val_loss: 0.9603 - val_acc: 0.6584
Epoch 7/50
Epoch 00007: val_loss improved from 0.37658 to 0.22879, saving model to best_m.h5
 - 2s - loss: 0.3766 - acc: 0.8454 - val_loss: 0.2288 - val_acc: 0.8978
Epoch 8/50
Epoch 00008: v

Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2617 - acc: 0.8923 - val_loss: 0.3390 - val_acc: 0.8329
Epoch 14/50
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2735 - acc: 0.8806 - val_loss: 0.3330 - val_acc: 0.8579
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 2s - loss: 0.2627 - acc: 0.8900 - val_loss: 0.2915 - val_acc: 0.8853
Epoch 16/50
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2870 - acc: 0.8731 - val_loss: 0.4602 - val_acc: 0.8130
Epoch 17/50
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.2519 - acc: 0.8817 - val_loss: 0.4872 - val_acc: 0.7980
Epoch 18/50
Epoch 00018: val_loss did not improve
 - 2s - loss: 0.2650 - acc: 0.8875 - val_loss: 0.2951 - val_acc: 0.8678
Epoch 19/50
Epoch 00019: val_loss improved from 0.25697 to 0.25531, saving model to best_m.h5
 - 2s - loss: 0.2117 - acc: 0.9133 - val_loss: 0.2553 - val_acc: 0.8878
Epoch 20/50
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.2263 - acc: 0.9117 - val_loss: 0.2836 - val_acc:

Epoch 28/50
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2803 - acc: 0.8639 - val_loss: 0.2318 - val_acc: 0.8953
Epoch 29/50
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2581 - acc: 0.8870 - val_loss: 0.5416 - val_acc: 0.8130
Epoch 30/50
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2371 - acc: 0.9050 - val_loss: 0.2474 - val_acc: 0.8953
Epoch 31/50
Epoch 00031: val_loss did not improve
 - 2s - loss: 0.1996 - acc: 0.9067 - val_loss: 0.2542 - val_acc: 0.8953
Epoch 32/50
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.2249 - acc: 0.9089 - val_loss: 0.2115 - val_acc: 0.9077
Epoch 33/50
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.1943 - acc: 0.9150 - val_loss: 0.2124 - val_acc: 0.9152
Epoch 34/50
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.2106 - acc: 0.9108 - val_loss: 0.2174 - val_acc: 0.9077
Epoch 35/50
Epoch 00035: val_loss did not improve
 - 2s - loss: 0.2212 - acc: 0.9042 - val_loss: 0.2388 - val_acc: 0.8953
Epoch 36/50
Epoch 00036:

Epoch 42/50
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.1571 - acc: 0.9308 - val_loss: 0.3221 - val_acc: 0.9027
Epoch 43/50
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.1770 - acc: 0.9131 - val_loss: 0.3625 - val_acc: 0.8903
Epoch 44/50
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.1547 - acc: 0.9308 - val_loss: 0.2982 - val_acc: 0.8928
Epoch 45/50
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.1708 - acc: 0.9317 - val_loss: 0.3303 - val_acc: 0.8903
Epoch 46/50
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.1748 - acc: 0.9383 - val_loss: 0.3495 - val_acc: 0.8803
Epoch 47/50
Epoch 00047: val_loss did not improve
 - 2s - loss: 0.1535 - acc: 0.9383 - val_loss: 0.3246 - val_acc: 0.9002
Epoch 48/50
Epoch 00048: val_loss did not improve
 - 2s - loss: 0.1779 - acc: 0.9181 - val_loss: 0.3096 - val_acc: 0.8853
Epoch 49/50
Epoch 00049: val_loss did not improve
 - 2s - loss: 0.1388 - acc: 0.9467 - val_loss: 0.3058 - val_acc: 0.8878
Epoch 50/50
Epoch 00050:

In [8]:
import pickle
with open('../features/cnn_4_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug2_sub.csv', index=False)

0.220099787017
         id  is_iceberg
0  5941774d    0.009409
1  4023181e    0.856684
2  b20200e4    0.000082
3  e7f018bb    0.995153
4  4371c8c3    0.003172
